In [19]:
import pandas as pd
import numpy as np
from datetime import datetime

In [20]:
df = pd.read_excel("C:\\Users\\gpavez\\Desktop\\Indemnización.xlsx")
df.head()

,RUT,Nombre Completo,Género,Fecha_Nacimiento,Fecha_Activacion,Fecha_Inicio_Contrato,Cargo,ID_Area,Nombre_Area,Primer_Nivel,Segundo_Nivel,Edad,Años de Servicio,Sueldo Base
0,4.775.647-2,Miguel Renato Berndt Cramer,M,1944-05-06,2016-06-01,2025-04-01,Directorio,511,Rol Privado,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,Administración,81.2,9.2,9800000
1,8.639.116-3,Victor Jose Tapia Valenzuela,M,1959-07-23,2017-03-23,2025-04-01,Operario,445,Fabricación Y Envasado De Fragancias,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,Operaciones,66.0,8.4,719000
2,7.773.489-9,Angel Reynaldo Cristi Cortes,M,1959-10-15,1987-06-04,2025-04-01,Operario,446,Fabricación Y Envasado De Esencias,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,Operaciones,65.8,38.2,1580000
3,8.270.763-8,Maria Soledad Yaryes Vergara,F,1960-03-18,1994-02-21,2025-04-01,Subgerente De Creacion Y Aplicacion De S,423,Desarrollo Sabores Salados,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,Sabores,65.4,31.4,5850000
4,8.961.105-9,David Riveros Guzman,M,1960-05-06,2007-06-04,2025-04-01,Operario,445,Fabricación Y Envasado De Fragancias,CARLOS CRAMER PRODUCTOS AROMÁTICOS S.A. C.I.,Operaciones,65.2,18.2,897000


In [21]:
df.columns

Index(['RUT', 'Nombre Completo', 'Género', 'Fecha_Nacimiento',
       'Fecha_Activacion', 'Fecha_Inicio_Contrato', 'Cargo', 'ID_Area',
       'Nombre_Area', 'Primer_Nivel', 'Segundo_Nivel', 'Edad',
       'Años de Servicio', 'Sueldo Base'],
      dtype='object')

In [22]:
df_limpio= df.drop(columns=['Género','Fecha_Nacimiento','Fecha_Activacion','Fecha_Inicio_Contrato','ID_Area','Nombre_Area','Primer_Nivel','Segundo_Nivel'], axis=0)
df_limpio = df_limpio.rename(columns={"Años de Servicio": "Anios de Servicio"})
df_limpio.columns


Index(['RUT', 'Nombre Completo', 'Cargo', 'Edad', 'Anios de Servicio',
       'Sueldo Base'],
      dtype='object')

In [24]:

def obtener_valor_uf():
    """Valor UF actualizado - verificar valor actual"""
    return 39190  # Actualizar con valor real de agosto 2025

def calcular_indemnizacion_empleado(sueldo_base, anos_servicio, edad=None, con_clausula_no_competencia=True):
    """Calcula indemnización según política Cramer"""
    
    if anos_servicio < 4:
        return {
            'elegible': False,
            'rango': 'No elegible',
            'indemnizacion_neta': 0,
            'razon': 'Menos de 4 años de servicio'
        }
    
    # Regla especial 65+ años
    if edad is not None and edad >= 65 and anos_servicio >= 4:
        # Aplicar condiciones Rango C
        if con_clausula_no_competencia:
            sueldo_con_tope = sueldo_base
            anos_para_calculo = anos_servicio
        else:
            valor_uf = obtener_valor_uf()
            tope_pesos = 90 * valor_uf
            sueldo_con_tope = min(sueldo_base, tope_pesos)
            anos_para_calculo = min(anos_servicio, 16)
        
        indemnizacion_bruta = sueldo_con_tope * anos_para_calculo
        descuento_cesantia = indemnizacion_bruta * 0.024
        indemnizacion_neta = indemnizacion_bruta - descuento_cesantia
        
        return {
            'elegible': True,
            'rango': 'C (65+ años)',
            'sueldo_con_tope': sueldo_con_tope,
            'anos_para_calculo': anos_para_calculo,
            'indemnizacion_bruta': indemnizacion_bruta,
            'descuento_cesantia': descuento_cesantia,
            'indemnizacion_neta': indemnizacion_neta,
            'regla_especial': True
        }
    
    # Clasificación normal por antigüedad
    valor_uf = obtener_valor_uf()
    tope_uf_90 = 90 * valor_uf
    
    if 4 <= anos_servicio < 20:
        # Rango A
        rango = 'A'
        sueldo_con_tope = min(sueldo_base, tope_uf_90)
        anos_para_calculo = min(anos_servicio, 11)
    elif 20 <= anos_servicio < 25:
        # Rango B
        rango = 'B'
        sueldo_con_tope = min(sueldo_base, tope_uf_90)
        anos_para_calculo = min(anos_servicio, 16)
    else:  # 25+ años
        # Rango C
        rango = 'C'
        if con_clausula_no_competencia:
            sueldo_con_tope = sueldo_base  # Sin tope
            anos_para_calculo = anos_servicio
        else:
            sueldo_con_tope = min(sueldo_base, tope_uf_90)
            anos_para_calculo = min(anos_servicio, 16)
    
    indemnizacion_bruta = sueldo_con_tope * anos_para_calculo
    descuento_cesantia = indemnizacion_bruta * 0.024
    indemnizacion_neta = indemnizacion_bruta - descuento_cesantia
    
    return {
        'elegible': True,
        'rango': rango,
        'sueldo_con_tope': sueldo_con_tope,
        'anos_para_calculo': anos_para_calculo,
        'indemnizacion_bruta': indemnizacion_bruta,
        'descuento_cesantia': descuento_cesantia,
        'indemnizacion_neta': indemnizacion_neta,
        'regla_especial': False
    }

# Aplicar cálculos al DataFrame
resultados = []
for index, row in df_limpio.iterrows():
    resultado = calcular_indemnizacion_empleado(
        sueldo_base=row['Sueldo Base'],
        anos_servicio=row['Anios de Servicio'],
        edad=row['Edad'],
        con_clausula_no_competencia=True  # Cambiar a False si no aceptan cláusula
    )
    resultados.append(resultado)

# Agregar columnas de resultado al DataFrame
df_limpio['Rango_Antiguedad'] = [r['rango'] for r in resultados]
df_limpio['Elegible'] = [r['elegible'] for r in resultados]
df_limpio['Indemnizacion_Neta'] = [r['indemnizacion_neta'] for r in resultados]
df_limpio['Sueldo_Con_Tope'] = [r.get('sueldo_con_tope', 0) for r in resultados]
df_limpio['Anos_Para_Calculo'] = [r.get('anos_para_calculo', 0) for r in resultados]
df_limpio['Regla_Especial_65'] = [r.get('regla_especial', False) for r in resultados]

# Mostrar resultados
print("=== RESUMEN GENERAL ===")
print(f"Total empleados: {len(df_limpio)}")
print(f"Empleados elegibles: {df_limpio['Elegible'].sum()}")
print(f"Total indemnizaciones: ${df_limpio['Indemnizacion_Neta'].sum():,.0f}")

print("\n=== RESUMEN POR RANGO ===")
resumen = df_limpio[df_limpio['Elegible']].groupby('Rango_Antiguedad').agg({
    'Elegible': 'count',
    'Indemnizacion_Neta': ['sum', 'mean', 'min', 'max']
}).round(0)
print(resumen)

print("\n=== EMPLEADOS CON MAYOR INDEMNIZACIÓN ===")
top_indemnizaciones = df_limpio[df_limpio['Elegible']].nlargest(10, 'Indemnizacion_Neta')[
    ['Nombre Completo', 'Cargo', 'Edad', 'Anios de Servicio', 'Sueldo Base', 'Rango_Antiguedad', 'Indemnizacion_Neta']
]
print(top_indemnizaciones)

# Mostrar algunos ejemplos específicos
print("\n=== EJEMPLOS ESPECÍFICOS ===")
ejemplos = df_limpio[df_limpio['Elegible']].head(5)[
    ['Nombre Completo', 'Edad', 'Anios de Servicio', 'Sueldo Base', 'Rango_Antiguedad', 'Indemnizacion_Neta']
]
print(ejemplos)

# Guardar resultados
df_limpio.to_excel("C:\\Users\\gpavez\\Desktop\\Indemnizacion_Calculada.xlsx", index=False)
print("\nResultados guardados en: Indemnizacion_Calculada.xlsx")

=== RESUMEN GENERAL ===
Total empleados: 44
Empleados elegibles: 44
Total indemnizaciones: $3,442,185,045

=== RESUMEN POR RANGO ===
                 Elegible Indemnizacion_Neta                           \
                    count                sum         mean         min   
Rango_Antiguedad                                                        
A                      18       2.637813e+08   14654517.0   4585248.0   
B                       5       1.969068e+08   39381366.0  14569728.0   
C                      16       2.633484e+09  164592725.0  35198464.0   
C (65+ años)            5       3.480133e+08   69602659.0   5894650.0   

                               
                          max  
Rango_Antiguedad               
A                  37866946.0  
B                  55079194.0  
C                 522238080.0  
C (65+ años)      179281440.0  

=== EMPLEADOS CON MAYOR INDEMNIZACIÓN ===
                    Nombre Completo  \
39     Miguel Andres Berndt Briceño   
30       M